In [113]:
import numpy as np
import pandas as pd

## Import Datasets

In [114]:
path = "Dataset"
pitstops = pd.read_csv(f"{path}/pit_stops.csv")
drivers = pd.read_csv(f"{path}/drivers.csv")
races = pd.read_csv(f"{path}/races.csv")
results = pd.read_csv(f"{path}/results.csv")
qualifying = pd.read_csv(f"{path}/qualifying.csv")
races = pd.read_csv(f"{path}/races.csv")


#### Race Results with Pitstop Information for Drivers Who Successfully Completed the Race

In [115]:
results_s = results.query("raceId >= 841 and ( statusId == 1 or (statusId >= 11 and statusId <= 19))")

#### Count the total number of pitstops for each race-driver pair

In [116]:
pitstops_count = pitstops.groupby(['raceId' , 'driverId']).count()['stop']

In [117]:
results_s = results_s.merge(pitstops_count,how="left" , on=['raceId' , 'driverId'])

results_s['pitstrategy'] = np.where(results_s['positionOrder'] <= results_s['grid'], 1 ,0 ) 

In [118]:
results_s.dropna(subset=['stop'], inplace=True)

In [119]:

first_10_in_grid = results_s.query('grid <= 10 and grid>0')

first_10_in_grid.dtypes

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
stop               float64
pitstrategy          int32
dtype: object

In [120]:
a = first_10_in_grid.drop(['number','resultId','points','laps','time','milliseconds','fastestLapTime','fastestLap','fastestLapSpeed','statusId','time','position','positionText','resultId'], axis=1)
a = a.merge(races[['raceId','circuitId','year']], how='left',on=['raceId'])
a = a.merge(qualifying[['raceId' , 'driverId','q1','q2','q3']],on=['raceId' , 'driverId'],how="left")

In [121]:

new_column_order = [col for col in a.columns if col != 'pitstrategy'] + ['pitstrategy'] 
a = a.reindex(columns=new_column_order)

In [122]:
count = [0 ,0 ,0 ]
for i in a.index.values:
    q3 = a.loc[i,'q3']
    if  ":" not in str(q3):
        a.loc[i,'q3'] = a.loc[i - 1,'q3']
        count[2] = count[2]+1
    q2 = a.loc[i,'q2']
    if ":" not in str(q2):
        a.loc[i,'q2'] = a.loc[i - 1,'q2']
        count[1] = count[1]+1
    q1 = a.loc[i,'q1']
    if ":" not in str(q1):
        a.loc[i,'q1'] = a.loc[i - 1,'q1']
        count[0] = count[0]+1
    


In [123]:
def convert_to_time(value):
    value = str(value)
    print(value )
    parts = value.split(':')
    parts1 = parts[1].split(".")
    
    minutes = int(parts[0])
    seconds = int(parts1[0])
    milliseconds = int(parts1[1])
    return 60*1000*minutes + 1000*seconds + milliseconds



In [124]:
for column in ['q3','q2','q1']:
    a[column] = a[column].apply(convert_to_time)

1:23.529
1:24.307
1:25.247
1:24.974
1:24.395
1:24.779
1:25.599
1:27.066
1:34.870
1:35.200
1:36.124
1:35.179
1:36.251
1:35.802
1:36.820
1:34.974
1:36.809
1:34.463
1:33.706
1:34.421
1:34.670
1:35.145
1:35.119
1:35.119
1:36.190
1:36.203
1:25.049
1:25.454
1:25.851
1:25.595
1:25.574
1:25.982
1:26.659
1:26.296
1:26.296
1:26.646
1:21.181
1:21.961
1:21.996
1:20.981
1:21.964
1:21.964
1:22.599
1:22.471
1:22.952
1:13.556
1:14.483
1:13.997
1:14.019
1:15.280
1:15.766
1:13.838
1:13.014
1:13.429
1:13.864
1:14.085
1:13.217
1:13.814
1:36.975
1:37.454
1:37.163
1:37.380
1:37.535
1:37.645
1:38.231
1:38.231
1:38.231
1:38.240
1:30.516
1:30.431
1:30.399
1:32.376
1:31.124
1:32.209
1:31.933
1:31.929
1:30.134
1:30.442
1:30.079
1:30.216
1:30.910
1:32.010
1:31.263
1:32.482
1:32.187
1:20.024
1:19.815
1:20.365
1:19.978
1:20.474
1:20.350
1:21.098
1:21.445
1:21.445
1:48.298
1:49.376
1:51.251
1:50.552
1:50.256
1:52.303
1:51.121
1:22.275
1:22.777
1:22.841
1:22.725
1:23.777
1:23.188
1:23.188
1:44.381
1:44.804
1:44.732
1

In [125]:

raceIds = np.unique(a['raceId'])

for raceId in raceIds:
    filtered_acc_race = a[a['raceId'] == raceId]
    min_q1 = np.min(filtered_acc_race['q1'])
    min_q2 = np.min(filtered_acc_race['q2'])
    min_q3 = np.min(filtered_acc_race['q3'])
    for i in filtered_acc_race.index.values:
        a.loc[i,'q1'] = a.loc[i, 'q1'] / min_q1
        a.loc[i,'q2'] = a.loc[i, 'q2'] / min_q2
        a.loc[i,'q3'] = a.loc[i, 'q3'] / min_q3

C:\Users\bpk_e\AppData\Local\Temp\ipykernel_19900\890089864.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.001031701369349' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  a.loc[i,'q1'] = a.loc[i, 'q1'] / min_q1
C:\Users\bpk_e\AppData\Local\Temp\ipykernel_19900\890089864.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.006005470329409' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  a.loc[i,'q2'] = a.loc[i, 'q2'] / min_q2
C:\Users\bpk_e\AppData\Local\Temp\ipykernel_19900\890089864.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0093141304217697' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  a.loc[i,

In [126]:
a['avg_finish'] = np.zeros(len(a))
for i in a.index.values:
    driverId , raceId = a.loc[i, 'driverId'] , a.loc[i, 'raceId']
    all_previous = results.query(f"driverId == {driverId} and  raceId < {raceId}")
    average_finish = 21
    if len(all_previous) != 0:
        average_finish = np.sum( all_previous['positionOrder'] ) / len(all_previous)
        a.loc[i,'avg_finish'] = average_finish

In [129]:
a['rank'] = a['rank'].astype('int64')

In [130]:
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance


In [131]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [133]:
params = {
    'objective': 'multi:softmax',  # Set objective to multi-class classification
    'num_class': 3 ,  # Number of classes
    'eval_metric': 'mlogloss',
    'eta': 0.1,  # Learning rate
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'seed': 42
}

In [140]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier


def best_pitstop_estimations_for_top_10(circuitId , params):
    b = a.query(f" (circuitId=={circuitId}) and stop<=3")
    y = b[['stop']] - 1 
    X = b.drop(columns=['pitstrategy','positionOrder','stop']) 
    
    X_train, X_test = train_test_split(X, train_size=0.67, random_state=42)
    y_train, y_test = train_test_split(y, train_size=0.67, random_state=42)
    
    X_train , X_val = train_test_split(X_train, train_size=0.67, random_state=42)
    y_train , y_val = train_test_split(y_train, train_size=0.67, random_state=42)
    
    # Convert data to DMatrix format
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = train_xgboost_model(X_train, y_train, X_val, y_val , params)
    
    y_pred_proba = model.predict(dtest)
    print(y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred_proba)
    print(f"Circuit ID {circuitId} --- accuracy : {accuracy}")
    conf_mat = np.array(tf.math.confusion_matrix(labels = y_test, predictions = y_pred_proba))
    print(conf_mat)
    return model
    
def train_xgboost_model(X_train, y_train, X_val, y_val ,params):
    xgb = XGBClassifier(eval_metric='mlogloss')
    grid_xgb = GridSearchCV(xgb, params, scoring='accuracy', cv=None, n_jobs=1)
    grid_xgb.fit(X_train, y_train)
    
    best_xgb_parameters = grid_xgb.best_params_
    
    print('Grid Search found the following optimal parameters: ')
    for param_name in sorted(best_xgb_parameters.keys()):
        print('%s: %r' % (param_name, best_xgb_parameters[param_name]))
    return xgb

In [143]:
search_params = {
        'min_child_weight': [0 , 1, 5, 10],
        'gamma': [0, 0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6 ,7] } 

best_pitstop_estimations_for_top_10(1 , search_params)

Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x00000135ADDC0CB0>>
Traceback (most recent call last):
  File "C:\Users\bpk_e\PycharmProjects\F1\.venv\Lib\site-packages\xgboost\core.py", line 641, in _next_wrapper
    return self._handle_exception(lambda: self.next(input_data), 0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bpk_e\PycharmProjects\F1\.venv\Lib\site-packages\xgboost\core.py", line 557, in _handle_exception
    return fn()
           ^^^^
  File "C:\Users\bpk_e\PycharmProjects\F1\.venv\Lib\site-packages\xgboost\core.py", line 641, in <lambda>
    return self._handle_exception(lambda: self.next(input_data), 0)
                                          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bpk_e\PycharmProjects\F1\.venv\Lib\site-packages\xgboost\data.py", line 1280, in next
    input_data(**self.kwargs)
  File "C:\Users\bpk_e\PycharmProjec

KeyboardInterrupt: 

In [104]:
final_model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=0, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [105]:
final_model_training_preds = final_model.predict(X_train)
final_model_test_preds = final_model.predict(X_test)

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 2,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1], dtype=int64)

AttributeError: module 'xgboost' has no attribute 'get_params'

In [0]:
# from tensorflow.python.keras.callbacks import EarlyStopping
# 
# # y = one_hot_encoded
# k_folds = 2
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=12)
# 
# def create_model():
#     model = tf.keras.Sequential([
#         Dense(64, activation='sigmoid'),
#         Dense(128, activation='sigmoid'),
#         Dense(1, activation='sigmoid')
#     ])
# 
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model
# 
# all_loss = []
# all_accuracy = []
# training_histories = []  
# 
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     
#     X_train , X_val = train_test_split(X_train, train_size=0.8, random_state=42, stratify=y_train)
#     y_train , y_val = train_test_split(y_train, train_size=0.8, random_state=42, stratify=y_train)
#     model = create_model()
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto',restore_best_weights=True)
# 
#     history = model.fit( X_train, y_train,validation_data = (X_val,y_val), epochs=200, batch_size=16, verbose=0, callbacks = [early_stopping] , class_weight = {0:class_weights[0],1:class_weights[1]})
# 
#     training_histories.append(history.history)
#     y_pred_proba = model.predict(X_test)
#     y_pred = [1 if pred > 5/10 else 0 for pred in y_pred_proba]
#     loss, accuracy = model.evaluate(X_test, y_test)
#     all_loss.append(loss)
#     all_accuracy.append(accuracy)
#     
#     # r = permutation_importance(model, X_val, y_val, n_repeats=30, random_state=0 ,verbose=0, scoring='neg_root_mean_squared_log_error')
#     # for i in r.importances_mean.argsort()[::-1]:
#     #     # if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#     #      print(f"{a.columns[i]:<8}"
#     #            f"{r.importances_mean[i]:.3f}"
#     #            f" +/- {r.importances_std[i]:.3f}")
#     #     
#     print(tf.math.confusion_matrix(labels = y_test, predictions = y_pred))
#     # print(r)
# avg_loss = np.mean(all_loss)
# avg_accuracy = np.mean(all_accuracy)
# 
# print(f'Average Loss: {avg_loss}, Average Accuracy: {avg_accuracy}')
# 
# # Plot training results
# plt.figure(figsize=(12, 24))
# 
# for fold, history in enumerate(training_histories):
#     # print(history)
#     plt.subplot(2, k_folds // 2, fold + 1)
#     plt.plot(history['accuracy'][::1], label='Training Accuracy')
#     plt.plot(history['val_accuracy'][::1], label='Validation Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.title(f'Fold {fold+1} Accuracy')
#     plt.legend()
# 
# plt.tight_layout()
# plt.show()

In [0]:
# from tensorflow.python.keras.callbacks import EarlyStopping
# 
# Y = y
# def create_model():
#     model = tf.keras.Sequential([
#         Dense(32),
#         Dense(64),
#         Dense( 1 , activation='softmax')
#     ])
#     opt = tf.keras.optimizers.Adam(0.001)
#     model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
#     return model
# 
# all_loss = []
# all_accuracy = []
# training_histories = []  
# 
# 
# X_train, X_test = train_test_split(X, train_size=0.8, random_state=42, )
# y_train, y_test = train_test_split(Y, train_size=0.8, random_state=42, )
# X_train , X_val = train_test_split(X_train, train_size=0.8, random_state=42, )
# y_train , y_val = train_test_split(y_train, train_size=0.8, random_state=42, )
# model = create_model()
# early_stopping = EarlyStopping(monitor='val_loss', patience=500, verbose=0, mode='auto',restore_best_weights=True)
# 
# history = model.fit( X_train, y_train,validation_data = (X_val,y_val), epochs=20, batch_size=8, verbose=2, callbacks = [early_stopping] )
# 
# # history = model.fit( X_train, y_train,validation_data = (X_val,y_val), epochs=20, batch_size=16, verbose=2, callbacks = [early_stopping] , class_weight = {0:0.4,1:1.2})
# # 
# 
# training_histories.append(history.history)
# y_pred_proba = model.predict(X_test)
# y_pred = [1 if pred > 5/10 else 0 for pred in y_pred_proba]
# # y_pred = [ np.argmax(i) for i in y_pred_proba]
# print(y_pred)
# loss, accuracy = model.evaluate(X_test, y_test)
# all_loss.append(loss)
# all_accuracy.append(accuracy)
# 
#     # r = permutation_importance(model, X_val, y_val, n_repeats=30, random_state=0 ,verbose=0, scoring='neg_root_mean_squared_log_error')
#     # for i in r.importances_mean.argsort()[::-1]:
#     #     # if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#     #      print(f"{a.columns[i]:<8}"
#     #            f"{r.importances_mean[i]:.3f}"
#     #            f" +/- {r.importances_std[i]:.3f}")
#     #     
# print(tf.math.confusion_matrix(labels = y_test, predictions = y_pred))
#     # print(r)
# avg_loss = np.mean(all_loss)
# avg_accuracy = np.mean(all_accuracy)
# 
# print(f'Average Loss: {avg_loss}, Average Accuracy: {avg_accuracy}')
# 
# # Plot training results
# plt.figure(figsize=(12, 24))
# 
# for fold, history in enumerate(training_histories):
#     # print(history)
#     plt.subplot(2, 1, fold + 1)
#     plt.plot(history['accuracy'][::1], label='Training Accuracy')
#     plt.plot(history['val_accuracy'][::1], label='Validation Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.title(f'Fold {fold+1} Accuracy')
#     plt.legend()
# 
# plt.tight_layout()
# plt.show()

In [0]:
# k_folds = 2
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=12)
# 
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.loc[train_index], X.loc[test_index]
#     y_train, y_test = y.loc[train_index], y.loc[test_index]
# 
#     X_train , X_val = train_test_split(X_train, train_size=0.8, random_state=42, stratify=y_train)
#     y_train , y_val = train_test_split(y_train, train_size=0.8, random_state=42, stratify=y_train)
# 
#     # Convert data to DMatrix format
#     dtest = xgb.DMatrix(X_test, label=y_test)
# 
#     model = train_xgboost_model(X_train, y_train, X_val, y_val)
# 
#     y_pred_proba = model.predict(dtest)
#     y_pred = [1 if pred > 0.5 else 0 for pred in y_pred_proba]
#     accuracy = accuracy_score(y_test, y_pred)
# 
#     print(f'Accuracy: {accuracy}')